In [1]:
%load_ext lab_black

In [2]:
import warnings

warnings.filterwarnings("ignore")

In [3]:
import os
import time
import numpy as np
import tushare as ts
import pandas as pd
from datetime import datetime, timedelta

In [4]:
import getpass

token = getpass.getpass("token:")
pro = ts.pro_api(token)

token: ························································


# 获取当天日期
```python
Parameters: None

Returns: str ('20190325')
```

In [5]:
get_date = lambda: time.strftime("%Y%m%d", time.localtime(time.time()))

# 获取前N天日期
```python
Parameters:
    
    date: str ('20190325') 默认今天
        
    N: int 前 N 天，默认 1 天

Returns: str ('20190324')
```

In [6]:
get_pastdate = lambda date=get_date(), N=1: datetime.strftime(
    pd.to_datetime(date) - timedelta(N + 1 - 1), "%Y%m%d"
)

# 获取前N后日期
```python
Parameters:
    
    date: str ('20190325') 默认今天
        
    N: int 后 N 天，默认 1 天

Returns: str ('20190324')
```

In [7]:
get_afterdate = lambda date=get_date(), N=1: datetime.strftime(
    pd.to_datetime(date) + timedelta(N + 1 - 1), "%Y%m%d"
)

# 获取最近N天单只个股的数据
```python
Parameters:
    
    code: str ('000001.SZ') 没有默认
        
    date: str ('20190325') 默认今天
        
    peroid: int 从 date 往前几天，默认 300 天

Returns: DataFrame 该支股票的历史数据
```

In [8]:
def get_oneNdays(code, date=get_date(), period=300):
    start_date = get_pastdate(date=date, N=period)
    name = pro.stock_basic(exchange="", list_status="L", fields="ts_code,name")
    df = pro.daily(ts_code=code, start_date=start_date, end_date=date).iloc[::-1]
    df["state"] = df["close"] >= df["open"]  # 阳柱:True, 阴柱:False
    df.index = range(len(df))
    df.name = "%s-%s" % (name[name["ts_code"] == code]["name"].item(), code)
    return df

# 凹口线

```python
判断凹口的函数，在 plotly_ohlc 中当输入为 draw_line={'凹口':} 时调用，因此不建议单独运行，也不设默认值。

前后 min_range 天的低点，往前 back_days 内找一天高点，当天涨幅 ('close'-'open') < increase，前后 max_range 天的 max('open','close') 不能超过当天的 max('open','close')。
    
Parameters:
    
    df: get_oneNdays()
        
    min_range: 找最低点时考虑前后各几项
        
    back_days: 从低点往前找的天数上限
        
    increase: 当天涨幅必须达到的标准
        
    max_range: 找最高点时考虑前后各几项

Returns: list 每个元素是一个 tuple，第一项是 index 第二项是 max('open','close')
```

In [9]:
def aokou(df, min_range, back_days, increase, max_range):
    df["min"] = (
        df["close"].rolling(2 * min_range + 1, min_periods=min_range, center=True).min()
    )
    bottom = []
    for i in range(len(df) - 1, 0, -1):
        if df.loc[i]["close"] == df.loc[i]["min"]:
            bottom.append(i)

    df["max"] = (
        df[["close", "open"]]
        .max(axis=1)
        .rolling(2 * max_range + 1, min_periods=max_range, center=True)
        .max()
    )
    top = []
    for i in range(len(df) - 1, 0, -1):
        if max(df.loc[i]["open"], df.loc[i]["close"]) == df.loc[i]["max"]:
            top.append(i)

    loc = []
    for btm in bottom:
        while top:
            tp = top.pop(0)
            if (
                tp < btm
                and (df.loc[tp]["close"] - df.loc[tp]["open"])
                < df.loc[tp]["open"] * increase
            ):
                loc.append((tp, df.loc[tp]["max"]))
                break
    return loc

# 黄金线

```python
判断黄金线（黄金柱柱顶）的函数，在 plotly_ohlc 中当输入为 draw_line={'黄金':} 时调用，因此不建议单独运行，也不设默认值。
    
Parameters:
    
    df: get_oneNdays()

Returns: list 每个元素是一个 tuple，第一项是 index 第二项是 max('open','close')
```

In [10]:
def huangjin(df):
    loc = []
    for i in range(len(df) - 2):
        o1 = df.loc[i]["open"]
        c1, c2, c3 = (
            df.loc[i]["close"],
            df.loc[i + 1]["close"],
            df.loc[i + 2]["close"],
        )
        v1, v2, v3 = (df.loc[i]["vol"], df.loc[i + 1]["vol"], df.loc[i + 2]["vol"])
        cond_open = o1 < min(c2, c3)  # 收盘价三日不破底
        cond_close = c1 < min(c2, c3)  # 收盘价三日不破顶
        cond_vol = (v1 > v2) & (v2 > v3)  # 量柱群三日不过头、不抬头
        if cond_open and cond_close and cond_vol:
            loc.append((i, max(df.loc[i]["open"], df.loc[i]["close"])))
    return loc

# 倍量线

```python
判断倍量线（倍量柱柱底）的函数，在 plotly_ohlc 中当输入为 draw_line={'倍量':} 时调用，因此不建议单独运行，也不设默认值。
    
Parameters:
    
    df: get_oneNdays()
        
    v_pct_min: float 今天的量比上昨天的量的最小值，默认 1.9
        
    v_pct_max: float 今天的量比上昨天的量的最大值，默认 2.1
        
    c_pct_min: float 今天的收盘价比上昨天的收盘价的最小值，默认 1.01

Returns: list 每个元素是一个 tuple，第一项是 index 第二项是 min('open','close')
```

In [11]:
def beiliang(df, v_pct_min, v_pct_max, c_pct_min):
    loc = []
    for i in range(len(df) - 1):
        yesterday_v = df.loc[i]["vol"]
        yesterday_c = df.loc[i]["close"]
        today_v = df.loc[i + 1]["vol"]
        today_c = df.loc[i + 1]["close"]
        v_pct = today_v / yesterday_v
        c_pct = today_c / yesterday_c
        cond1 = v_pct > v_pct_min and v_pct < v_pct_max
        cond2 = c_pct > c_pct_min
        if cond1 and cond2:
            loc.append((i + 1, min(df.loc[i + 1]["open"], df.loc[i + 1]["close"])))
    return loc

# Test

In [12]:
period = (
    pd.to_datetime(get_date()) - pd.to_datetime(get_pastdate(get_date(), 365))
).days
df = get_oneNdays("600866.SH", date=get_date(), period=period)

In [13]:
df.tail()

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount,state
230,600866.SH,20190408,6.62,6.97,6.41,6.87,6.34,0.53,8.3596,480159.56,325446.564,True
231,600866.SH,20190409,6.70,6.80,6.44,6.57,6.87,-0.30,-4.3668,240283.98,158382.540,False
232,600866.SH,20190410,6.58,6.75,6.40,6.55,6.57,-0.02,-0.3044,209609.58,137618.526,False
233,600866.SH,20190411,6.50,6.54,6.10,6.36,6.55,-0.19,-2.9008,244028.48,154485.180,False
234,600866.SH,20190412,6.35,6.73,6.28,6.66,6.36,0.30,4.7170,170581.71,110468.135,True


In [29]:
draw_line = {"凹口": 365}
lines = {}
length = {}

if "凹口" in draw_line.keys():
    lines["凹口"] = aokou(df, min_range=7, back_days=365, increase=0.09, max_range=7)
    length["凹口"] = draw_line["凹口"]

line = "凹口"
x_data, y_data = [], []
for date, price in lines[line]:
    from_date = df.loc[date]["trade_date"]
    to_date = min(
        pd.to_datetime(get_date()),
        pd.to_datetime(get_afterdate(from_date, length[line])),
    )
    x_data += list(pd.date_range(from_date, to_date))
    y_data += [price] * len(pd.date_range(from_date, to_date))

##### 正负5%的范围内只取最新的线 #####
data = {}
for x, y in zip(x_data, y_data):
    if x in data:
        data[x] += [y]
    else:
        data[x] = [y]
x_data_new, y_data_new, remove = [], [], []
for date in data:
    if len(data[date]) > 1:
        i = 0
        for i in range(len(data[date])):
            for j in range(i + 1, len(data[date])):
                if (
                    abs(data[date][i] - data[date][j]) < data[date][i] * 0.05
                    and data[date][j] not in remove
                ):
                    remove.append(data[date][j])
    remove = [k for k in remove if k in data[date]]
    add_line = list(set(data[date]) - set(remove))
    x_data_new += [date] * len(add_line)
    y_data_new += add_line

##### 正负5%的范围内只取最新的线 #####

df[line] = list(np.repeat(np.array([[]]), len(df), axis=0))
for x, y in zip(x_data_new, y_data_new):
    x_str = x.strftime("%Y%m%d")
    if x_str in list(df["trade_date"]):
        index = df[df["trade_date"] == x_str].index.item()
        if y not in df.at[index, line]:
            if line == "凹口" and y < max(df.at[index, "open"], df.at[index, "close"]):
                pass
            else:
                df.at[index, line] = np.append(df.at[index, line], y)

In [30]:
df["倍量"] = [0.0] * len(df)
for i, p in beiliang(df, v_pct_min=1.9, v_pct_max=2.1, c_pct_min=1.01):
    df.at[i, "倍量"] = p

In [31]:
df["黄金"] = [0.0] * len(df)
for i, p in huangjin(df):
    df.at[i, "黄金"] = p

In [32]:
in_aoukou = False
start, end = 0, 0
bl, hj = [], []
aoukou_line = []
print("Date\t\t\t凹口\tmax\t突破\t倍量\t黄金")
for i in range(len(df) - 1):
    if not in_aoukou and len(df.loc[i, "凹口"]) > 0:
        in_aoukou = True
        if len(aoukou_line) == 0:
            aoukou_line = df.loc[i, "凹口"]
        start = i
        end = 0
        bl, hj = [], []
    elif in_aoukou:
        if max(df.loc[i, "open"], df.loc[i, "close"]) > min(aoukou_line):
            in_aoukou = False
            end = i
            print(
                "%s-%s\t%5.2f\t%s\t%s\t%s\t%s"
                % (df.loc[start]['trade_date'], df.loc[end]['trade_date'], min(aoukou_line), max(df.loc[end, "open"], df.loc[end, "close"]),(min(aoukou_line)<max(df.loc[end,"open"],df.loc[i,"close"])),len(bl),len(hj))
            )
            aoukou_line = np.delete(aoukou_line, aoukou_line.argmin())
        elif set(df.loc[i, "凹口"]) != set(aoukou_line):
            aoukou_line = df.loc[i, "凹口"]
            start = i
            end = 0
            bl, hj = [], []
    if df.loc[i, "倍量"] not in bl + [0]:
        bl.append(df.loc[i, "倍量"])
    if df.loc[i, "黄金"] not in hj + [0]:
        hj.append(df.loc[i, "黄金"])

Date			凹口	max	突破	倍量	黄金
20190118-20190213	 3.39	3.41	True	0	0
20190214-20190306	 4.06	4.13	True	0	1
20190307-20190329	 5.12	5.36	True	1	0
